## This notebook requires a https://github.com/cocktailpeanut/dalai dalai server running

**Download and unpack dalaipy**

In [4]:
import os
processing_path = "../../data/question_answering/"
os.makedirs(processing_path, exist_ok=True)

!wget https://github.com/wastella/dalaipy/archive/refs/heads/main.zip -P {processing_path}
unzip_path_extractor = processing_path + "main.zip"
!unzip {unzip_path_extractor} -d {processing_path}

--2023-04-19 09:59:21--  https://github.com/wastella/dalaipy/archive/refs/heads/main.zip
Auflösen des Hostnamens github.com (github.com)… 140.82.121.4
Verbindungsaufbau zu github.com (github.com)|140.82.121.4|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 302 Found
Platz: https://codeload.github.com/wastella/dalaipy/zip/refs/heads/main [folgend]
--2023-04-19 09:59:21--  https://codeload.github.com/wastella/dalaipy/zip/refs/heads/main
Auflösen des Hostnamens codeload.github.com (codeload.github.com)… 140.82.121.10
Verbindungsaufbau zu codeload.github.com (codeload.github.com)|140.82.121.10|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: nicht spezifiziert [application/zip]
Wird in »../../data/question_answering/main.zip« gespeichert.

main.zip                [ <=>                ]  14,80K  --.-KB/s    in 0,02s   

2023-04-19 09:59:21 (859 KB/s) - »../../data/question_answering/main.zip« gespeichert [15160]

Archive:  ../..

**Move main file to current directory and import it**

In [1]:
!cp ../../data/question_answering/dalaipy-main/src/main.py daiaipy.py
from  daiaipy import Dalai
model = Dalai()

**function used for question generation**

In [25]:
import time 
def get_question(request):
    # used for testing
    # request = request[:40]
    # used for testing

    request_sentence ="The generated question should be answerable only by the information provided in the sentence. Fill the last field:\nAnswer: The capital of France is Paris. Generated question: What is the capital of France?\nAnswer: Green Dragon: The Green Dragon was a British pub. Generated question:What was the Green Dragon?\nAnswer: The movie was directed by Steven Spielberg. Generated question: Who directed the movie?\nAnswer: The temperature is 25 degrees Celsius. Generated question: What is the temperature in Celsius?\nAnswer: "
    request_sentence = request_sentence + request + " Generated question:"
    #request_sentence = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\n>Ask a question that can be answered by the following sentences and tell me what the answer in the sentence is.:\n### Input:\n>"
    #request_sentence = request_sentence + request + "\n### Response:"

    request_dict = model.generate_request(request_sentence, "alpaca.7B")
    str = model.generate(request_dict)['response'].replace("\r", "")[len(request_sentence):-7]
    time.sleep(4)
    return str.split("?")[0] + "?"

In [21]:
get_question("Lobalug: The Lobalug was an aquatic magical beast found at the bottom of the North Sea.")

' Where did you find the Lobalusu ?'

In [10]:
import pandas as pd
df = pd.read_pickle('../../data/dataframes/harrypotter_pages_current_cleaned.pickle')
df

,URL,text
0,http://harrypotter.fandom.com/wiki?curid=8220,"Advanced Rune Translation: Advanced Rune Translation was a book about Rune Translation by Yuri Blishen. It was a required textbook for Study of Ancient Runes, an elective course at Hogwarts School of Witchcraft and Wizardry."
1,http://harrypotter.fandom.com/wiki?curid=8220,"Advanced Rune Translation, History: Hermione Granger was reading a copy of this book after her trip to Diagon Alley before the start of her sixth year at Hogwarts."
2,http://harrypotter.fandom.com/wiki?curid=8218,"Murtlap: The Murtlap was a magical marine beast resembling a rat with a growth on its back resembling a sea anemone, found on the coastal areas of Britain."
3,http://harrypotter.fandom.com/wiki?curid=8218,"Murtlap, Nature: The favoured prey of the Murtlap were crustaceans, though they also went for the feet of any human foolish enough to step on them."
4,http://harrypotter.fandom.com/wiki?curid=8218,"Murtlap, Uses: The growth on the Murtlap's back may be pickled and eaten to improve one's resistance to jinxes, although eating an excess of pickled murtlap may cause one to grow unsightly purple ear hair. Murtlap Essence was a home remedy for cuts and abrasions. Murtlap tentacles were included ..."
...,...,...
24764,http://harrypotter.fandom.com/wiki?curid=21665,Model of a Firebolt: A model of a Firebolt was a small working model of the world-class Firebolt. Harry Potter received one of these at Christmas from Nymphadora Tonks on 25 December 1995. Harry missed his real Firebolt which he got back at the end of the year while watching it zoom around his b...
24765,http://harrypotter.fandom.com/wiki?curid=21664,"Bewitched tea set: A bewitched tea set was owned by an old witch, until she died and the tea set was sold to a Muggle antique shop. It was subsequently purchased by a Muggle woman who used it for a tea party, but the bewitched tea set caused it to end in disaster and several injuries; the teapot..."
24766,http://harrypotter.fandom.com/wiki?curid=21660,"Intruder Charm: The Intruder Charm (""incantation unknown"") was a charm which detected intruders and sounded an alarm, the magical-equivalent to a burglar alarm."
24767,http://harrypotter.fandom.com/wiki?curid=21660,"Intruder Charm, History: Horace Slughorn used it on a Muggle-owned house he stayed in temporarily in 1996 in Budleigh Babberton, but did not hear it go off when Albus Dumbledore and Harry Potter arrived because he was in the bath."


In [11]:
from transformers import T5Config, T5ForConditionalGeneration, T5Tokenizer

model_name = "allenai/t5-small-squad2-question-generation"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    return output[0]

In [12]:
df['questions'] = df['text'].apply(run_model)
df

/Users/tilldirschnabel/Library/Python/3.9/lib/python/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (859 > 512). Running this sequence through the model will result in indexing errors


,URL,text,questions
0,http://harrypotter.fandom.com/wiki?curid=8220,"Advanced Rune Translation: Advanced Rune Translation was a book about Rune Translation by Yuri Blishen. It was a required textbook for Study of Ancient Runes, an elective course at Hogwarts School of Witchcraft and Wizardry.",What was the required textbook for Study of Ancient Runes?
1,http://harrypotter.fandom.com/wiki?curid=8220,"Advanced Rune Translation, History: Hermione Granger was reading a copy of this book after her trip to Diagon Alley before the start of her sixth year at Hogwarts.",What was Granger's sixth year at Hogwarts?
2,http://harrypotter.fandom.com/wiki?curid=8218,"Murtlap: The Murtlap was a magical marine beast resembling a rat with a growth on its back resembling a sea anemone, found on the coastal areas of Britain.",What was the name of the marine beast found on Britain's coastal areas?
3,http://harrypotter.fandom.com/wiki?curid=8218,"Murtlap, Nature: The favoured prey of the Murtlap were crustaceans, though they also went for the feet of any human foolish enough to step on them.",What did the favored prey of the Murtlap go for?
4,http://harrypotter.fandom.com/wiki?curid=8218,"Murtlap, Uses: The growth on the Murtlap's back may be pickled and eaten to improve one's resistance to jinxes, although eating an excess of pickled murtlap may cause one to grow unsightly purple ear hair. Murtlap Essence was a home remedy for cuts and abrasions. Murtlap tentacles were included ...",What was the home remedy for cuts and abrasions?
...,...,...,...
24764,http://harrypotter.fandom.com/wiki?curid=21665,Model of a Firebolt: A model of a Firebolt was a small working model of the world-class Firebolt. Harry Potter received one of these at Christmas from Nymphadora Tonks on 25 December 1995. Harry missed his real Firebolt which he got back at the end of the year while watching it zoom around his b...,What was the name of the model of a Firebolt?
24765,http://harrypotter.fandom.com/wiki?curid=21664,"Bewitched tea set: A bewitched tea set was owned by an old witch, until she died and the tea set was sold to a Muggle antique shop. It was subsequently purchased by a Muggle woman who used it for a tea party, but the bewitched tea set caused it to end in disaster and several injuries; the teapot...",What was the name of the bewitched tea set?
24766,http://harrypotter.fandom.com/wiki?curid=21660,"Intruder Charm: The Intruder Charm (""incantation unknown"") was a charm which detected intruders and sounded an alarm, the magical-equivalent to a burglar alarm.",What was the magical-equivalent to a burglar alarm?
24767,http://harrypotter.fandom.com/wiki?curid=21660,"Intruder Charm, History: Horace Slughorn used it on a Muggle-owned house he stayed in temporarily in 1996 in Budleigh Babberton, but did not hear it go off when Albus Dumbledore and Harry Potter arrived because he was in the bath.",What was Horace Slughorn's home in 1996?


In [17]:

import numpy as np
pd.set_option('display.max_colwidth', 300)
for i in np.random.randint(len(df), size= 10):
    print(str(i) + " - " +  ": ")
    print(df.iloc[i]["text"])
    print("-----------question----------------")
    print(df.iloc[i]["questions"])
    print("------------------------------------------")



13779 - : 
House of Black, Family history: Early history.
-----------question----------------
House of Black, Family history: Early history.
------------------------------------------
690 - : 
Hogwarts Castle, Sixth floor: Sixth-floor corridor.
-----------question----------------
What is the Sixth floor of Hogwarts Castle?
------------------------------------------
22111 - : 
Angelica (disambiguation), Behind the scenes: Real-world individuals with the given name Angelica include:
-----------question----------------
What is the name of Angelica?
------------------------------------------
8212 - : 
Purple flames, History: After entering the chamber, purple flames sealed off the entrance, while black flames blocked the exit. Only by solving Severus Snape's Potion riddle could one leave the room. The last bottle on the right side contained a potion that allowed the drinker to pass unharmed through the flames and travel back the way they came.
-----------question----------------
What was t